In [1]:
import os
import cv2
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import shutil

In [2]:
def extract_page_info(filename):
    """Extract page number from a filename like 'measure_001_page1.png'"""
    page_match = re.search(r'page(\d+)', filename)
    if page_match:
        return int(page_match.group(1))
    return None

In [3]:
def get_sheet_music_files(directory):
    """Find all sheet music PNG files in a directory"""
    sheet_files = []
    for filename in os.listdir(directory):
        if filename.lower().endswith('.png') and not filename.startswith('measure_'):
            sheet_files.append(filename)
    return sheet_files

In [4]:
def match_sheet_to_page(sheet_filename, page_number):
    """Check if a sheet music filename corresponds to a specific page number."""
    base_name = os.path.splitext(sheet_filename)[0]
    page_indicators = [f"_{page_number}", f"-{page_number}"]
    
    for indicator in page_indicators:
        if base_name.endswith(indicator):
            return True
    
    if page_number == 1 and not any(base_name.endswith(f"_{i}") or base_name.endswith(f"-{i}") 
                                   for i in range(1, 10)):
        return True
    
    return False

In [5]:
def visualize_detailed_ordering(full_sheet_path, matches, output_path=None):
    """Create a visualization showing the exact ordering of measures."""
    full_sheet_img = cv2.imread(full_sheet_path)
    if full_sheet_img is None:
        print(f"Could not read full sheet music image: {full_sheet_path}")
        return
    
    # Extract original measure numbers for reference
    orig_numbers = {}
    for match in matches:
        filename = match['filename']
        num_match = re.search(r'measure_(\d+)_', filename)
        if num_match:
            orig_numbers[filename] = int(num_match.group(1))
    
    # Draw measures with ordering information
    for i, match in enumerate(matches, 1):
        top_left = match['top_left']
        w, h = match['width'], match['height']
        bottom_right = (top_left[0] + w, top_left[1] + h)
        
        # Draw rectangle with row-specific colors
        row_colors = [
            (0, 255, 0),    # Green
            (0, 200, 255),  # Yellow
            (0, 128, 255),  # Orange
            (0, 0, 255),    # Red
            (255, 0, 255),  # Purple
            (255, 0, 0)     # Blue
        ]
        color = row_colors[match['row'] % len(row_colors)]
        cv2.rectangle(full_sheet_img, top_left, bottom_right, color, 2)
        
        # Draw ordering information
        orig_num = orig_numbers.get(match['filename'], '?')
        
        # Draw sequential number (new ordering)
        cv2.putText(full_sheet_img, f"{i}", 
                   (top_left[0] + 5, top_left[1] + 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Draw original number and scale if not 1.0
        scale_text = f" s:{match['scale']:.1f}" if 'scale' in match and match['scale'] != 1.0 else ""
        cv2.putText(full_sheet_img, f"({orig_num}{scale_text})", 
                   (top_left[0] + 5, top_left[1] + 45),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        
        # Draw row number
        cv2.putText(full_sheet_img, f"r{match['row']}", 
                   (top_left[0] + w - 30, top_left[1] + 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    # Convert from BGR to RGB for matplotlib
    rgb_img = cv2.cvtColor(full_sheet_img, cv2.COLOR_BGR2RGB)
    
    # Display or save the image
    plt.figure(figsize=(15, 15))
    plt.imshow(rgb_img)
    plt.title(f"Measure Ordering (New Number with Original in Parentheses)")
    plt.axis('off')
    
    if output_path:
        plt.savefig(output_path)
        print(f"Visualization saved to {output_path}")
    else:
        plt.show()
    
    plt.close()

In [6]:
def multi_scale_template_match(full_sheet_img, measure_img, max_scale=2.5, min_scale=0.1, scale_steps=75):
    """
    Perform template matching with multiple scales to handle different sized measures.
    
    Args:
        full_sheet_img: Full sheet music image
        measure_img: Measure image to match
        max_scale: Maximum scaling factor
        min_scale: Minimum scaling factor
        scale_steps: Number of scaling steps to try
        
    Returns:
        best_match: Dictionary with match information
    """
    best_match = {
        'confidence': 0,
        'top_left': None,
        'scale': 1.0,
        'width': measure_img.shape[1],
        'height': measure_img.shape[0]
    }
    
    # Try exact match first (scale = 1.0)
    result = cv2.matchTemplate(full_sheet_img, measure_img, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    
    # If we get a perfect or near-perfect match, return immediately
    if max_val > 0.7:
        best_match['confidence'] = max_val
        best_match['top_left'] = max_loc
        return best_match
    
    # Update best match
    if max_val > best_match['confidence']:
        best_match['confidence'] = max_val
        best_match['top_left'] = max_loc
    
    # Try different scales
    scales = np.linspace(min_scale, max_scale, scale_steps)
    for scale in scales:
        # Skip scale 1.0 as we already tried it
        if scale == 1.0:
            continue
            
        # Resize the measure image
        width = int(measure_img.shape[1] * scale)
        height = int(measure_img.shape[0] * scale)
        if width <= 0 or height <= 0:
            continue
            
        resized_measure = cv2.resize(measure_img, (width, height), interpolation=cv2.INTER_AREA)
        
        # Perform template matching
        try:
            result = cv2.matchTemplate(full_sheet_img, resized_measure, cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            
            # Update best match if this is better
            if max_val > best_match['confidence']:
                best_match['confidence'] = max_val
                best_match['top_left'] = max_loc
                best_match['scale'] = scale
                best_match['width'] = width
                best_match['height'] = height
        except cv2.error:
            # Skip if there's an error (e.g., template larger than image)
            continue
    
    return best_match

In [7]:
def process_song_folder(song_folder, create_visualization=True, row_threshold=100):
    """
    Process a single song folder containing sheet music and measures.
    
    Args:
        song_folder: Path to the song folder
        create_visualization: Whether to create visualization images
        row_threshold: Distance threshold for grouping measures into rows (larger = more measures in same row)
    """
    print(f"\nProcessing song folder: {os.path.basename(song_folder)}")
    
    # Find the measures subfolder
    measures_dir = os.path.join(song_folder, "ordered_measures")
    if not os.path.exists(measures_dir):
        print(f"No measures folder found in {song_folder}, skipping.")
        return False
    
    # Create output directory for ordered measures
    output_dir = os.path.join(song_folder, "final_measures")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")
    
    # Get all measure image files
    measure_files = [f for f in os.listdir(measures_dir) 
                   if f.lower().endswith('.png') and 'measure_' in f]
    
    if not measure_files:
        print(f"No measure files found in {measures_dir}, skipping.")
        return False
    
    # Group measure files by page
    measures_by_page = {}
    for filename in measure_files:
        page_number = extract_page_info(filename)
        if page_number is not None:
            if page_number not in measures_by_page:
                measures_by_page[page_number] = []
            measures_by_page[page_number].append(filename)
    
    # Get all sheet music files
    sheet_files = get_sheet_music_files(song_folder)
    
    if not sheet_files:
        print(f"No sheet music PNG files found in {song_folder}, skipping.")
        return False
    
    all_matches = []
    
    # Create output directory for visualizations
    if create_visualization:
        viz_dir = os.path.join(song_folder, "new_visualizations")
        if not os.path.exists(viz_dir):
            os.makedirs(viz_dir)
    
    # Process each page
    for page_number, page_measures in measures_by_page.items():
        # Find corresponding sheet music file for this page
        sheet_file = None
        for f in sheet_files:
            if match_sheet_to_page(f, page_number):
                sheet_file = f
                break
        
        if not sheet_file:
            print(f"Warning: No matching sheet music found for page {page_number}.")
            continue
        
        full_sheet_path = os.path.join(song_folder, sheet_file)
        print(f"Processing page {page_number} with sheet music: {sheet_file}")
        
        # Load the full sheet music image
        full_sheet_img = cv2.imread(full_sheet_path, cv2.IMREAD_GRAYSCALE)
        if full_sheet_img is None:
            print(f"Could not read sheet music image: {full_sheet_path}")
            continue
        
        matches = []
        
        # Process each measure image for this page
        for filename in page_measures:
            filepath = os.path.join(measures_dir, filename)
            try:
                measure_img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
                
                if measure_img is None:
                    print(f"Could not read measure image: {filepath}")
                    continue
                
                # Use multi-scale template matching to handle different sized measures
                match_result = multi_scale_template_match(
                    full_sheet_img, 
                    measure_img
                )
                
                if match_result['confidence'] < 0.5:
                    print(f"Warning: Low match confidence ({match_result['confidence']:.2f}) for {filename}")
                    continue
                
                top_left = match_result['top_left']
                h, w = match_result['height'], match_result['width']
                center_x = top_left[0] + w // 2
                center_y = top_left[1] + h // 2
                
                matches.append({
                    'filename': filename,
                    'top_left': top_left,
                    'center': (center_x, center_y),
                    'confidence': match_result['confidence'],
                    'width': w,
                    'height': h,
                    'scale': match_result['scale'],
                    'page': page_number
                })
                
                scale_info = f" (scale: {match_result['scale']:.2f})" if match_result['scale'] != 1.0 else ""
                print(f"Matched {filename} at position {top_left} with confidence {match_result['confidence']:.2f}{scale_info}")
                
            except Exception as e:
                print(f"Error processing {filename}: {e}")
        
        if not matches:
            continue
            
        # Get sheet height to calculate relative positions
        sheet_height = full_sheet_img.shape[0]
        
        # Perform row detection with DBSCAN
        y_coords = np.array([m['center'][1] for m in matches]).reshape(-1, 1)
        clustering = DBSCAN(eps=row_threshold, min_samples=1).fit(y_coords)
        
        # Calculate median y-coordinate for each cluster
        cluster_labels = set(clustering.labels_)
        cluster_medians = {}
        for label in cluster_labels:
            indices = [i for i, x in enumerate(clustering.labels_) if x == label]
            y_values = [matches[i]['center'][1] for i in indices]
            cluster_medians[label] = np.median(y_values)
        
        # Sort clusters by median y-coordinate (top to bottom)
        sorted_labels = sorted(cluster_labels, key=lambda l: cluster_medians[l])
        
        # Map cluster label to row number
        label_to_row = {label: row for row, label in enumerate(sorted_labels)}
        
        # Assign rows to measures
        for match in matches:
            idx = matches.index(match)
            cluster_label = clustering.labels_[idx]
            match['row'] = label_to_row[cluster_label]
            
            # Normalize vertical position as percentage of page height
            match['y_pct'] = match['center'][1] / sheet_height
        
        # Group measures by row
        rows = {}
        for match in matches:
            row = match['row']
            if row not in rows:
                rows[row] = []
            rows[row].append(match)
        
        # Sort rows by vertical position
        sorted_rows = sorted(rows.keys())
        
        # Sort each row by x-coordinate only
        for row in sorted_rows:
            rows[row].sort(key=lambda m: m['center'][0])
        
        # Flatten rows back into a single list in reading order
        ordered_matches = []
        for row in sorted_rows:
            ordered_matches.extend(rows[row])
        
        # Create visualization if requested
        if create_visualization:
            viz_path = os.path.join(viz_dir, f"ordering_page{page_number}.png")
            visualize_detailed_ordering(full_sheet_path, ordered_matches, viz_path)
        
        all_matches.extend(ordered_matches)
    
    # Sort all matches by page number
    all_matches.sort(key=lambda m: m['page'])
    
    # Copy measures to output directory with new names
    measure_count = 1
    copied_count = 0
    
    # Track which files have been copied to avoid duplicates
    copied_files = set()
    
    for match in all_matches:
        source_path = os.path.join(measures_dir, match['filename'])
        page_part = f"page{match['page']}"
        
        # Extract current measure number for debugging
        current_num_match = re.search(r'measure_(\d+)_', match['filename'])
        current_num = current_num_match.group(1) if current_num_match else "unknown"
        
        # Create new filename with updated measure number but keeping the page info
        new_filename = f"measure_{measure_count:03d}_{page_part}.png"
        dest_path = os.path.join(output_dir, new_filename)
        
        # Check if we already have this destination file to avoid overwrites
        if dest_path in copied_files:
            print(f"Warning: Destination {new_filename} already exists. Skipping duplicate.")
            continue
        
        try:
            # Copy instead of rename
            shutil.copy2(source_path, dest_path)
            copied_files.add(dest_path)
            print(f"Copied measure_{current_num}_{page_part}.png to {new_filename}")
            copied_count += 1
        except Exception as e:
            print(f"Error copying {match['filename']}: {e}")
        
        measure_count += 1
    
    print(f"Finished processing song folder: {os.path.basename(song_folder)}")
    print(f"Total measures: {len(all_matches)}, Copied: {copied_count}")
    print(f"Ordered measures saved to: {output_dir}")
    
    return True

In [8]:
def batch_process_songs(root_folder, create_visualization=True, row_threshold=100):
    """Process all song folders within the root folder."""
    print(f"Starting batch processing in: {root_folder}")
    print(f"Using row threshold: {row_threshold}px")
    
    # Get all subdirectories (song folders)
    song_folders = []
    for item in os.listdir(root_folder):
        item_path = os.path.join(root_folder, item)
        if os.path.isdir(item_path):
            song_folders.append(item_path)
    
    if not song_folders:
        print("No song folders found.")
        return
    
    print(f"Found {len(song_folders)} song folders.")
    
    # Process each song folder
    successful = 0
    for song_folder in song_folders:
        if process_song_folder(song_folder, create_visualization, row_threshold):
            successful += 1
    
    print(f"\nBatch processing complete. Successfully processed {successful} out of {len(song_folders)} song folders.")


In [9]:
root_dir = 'measures_pngs'

# Process all song folders
batch_process_songs(root_dir, row_threshold=80)


Starting batch processing in: measures_pngs
Using row threshold: 80px
Found 220 song folders.

Processing song folder: How-Insensitive
Created output directory: measures_pngs/How-Insensitive/final_measures
Processing page 1 with sheet music: How-Insensitive_1.png
Matched measure_017_page1.png at position (1913, 1527) with confidence 1.00
Matched measure_003_page1.png at position (769, 583) with confidence 1.00
Matched measure_022_page1.png at position (277, 2258) with confidence 1.00
Matched measure_029_page1.png at position (1966, 2511) with confidence 1.00
Matched measure_008_page1.png at position (1334, 899) with confidence 1.00
Matched measure_024_page1.png at position (1392, 2239) with confidence 1.00
Matched measure_030_page1.png at position (308, 2940) with confidence 1.00
Matched measure_011_page1.png at position (797, 1279) with confidence 1.00
Matched measure_005_page1.png at position (1873, 557) with confidence 1.00
Matched measure_016_page1.png at position (1365, 1565) with

In [2]:
import os
import re

base_folder = 'measures_pngs'
pattern = re.compile(r"measure_(\d+)_page1\.png")

for song_folder in os.listdir(base_folder):
    final_measures_path = os.path.join(base_folder, song_folder, 'final_measures')
    
    if not os.path.isdir(final_measures_path):
        continue

    files = [f for f in os.listdir(final_measures_path) if pattern.match(f)]
    files.sort(key=lambda f: int(pattern.match(f).group(1)))

    new_names = {}
    for idx, filename in enumerate(files):
        new_number = idx + 1
        new_filename = f"measure_{new_number:03d}_page1.png"
        new_names[filename] = new_filename

    for old_name, new_name in new_names.items():
        temp_name = f"tmp_{new_name}"
        os.rename(os.path.join(final_measures_path, old_name), os.path.join(final_measures_path, temp_name))

    for old_name, new_name in new_names.items():
        temp_name = f"tmp_{new_name}"
        os.rename(os.path.join(final_measures_path, temp_name), os.path.join(final_measures_path, new_name))

    print(f"Processed: {final_measures_path}")


Processed: measures_pngs/How-Insensitive/final_measures
Processed: measures_pngs/Prelude-to-a-Kiss/final_measures
Processed: measures_pngs/Do-You-Know-What-It-Means-To-Means-New-Orleans/final_measures
Processed: measures_pngs/I-Mean-You/final_measures
Processed: measures_pngs/Early-Autumn/final_measures
Processed: measures_pngs/I-Get-a-Kick-Out-of-You/final_measures
Processed: measures_pngs/Minor-Mood/final_measures
Processed: measures_pngs/When-I-fall-in-love/final_measures
Processed: measures_pngs/Careless-Love/final_measures
Processed: measures_pngs/Cherokee/final_measures
Processed: measures_pngs/Anthropology/final_measures
Processed: measures_pngs/Lady-Bird/final_measures
Processed: measures_pngs/Take-The-A-Train/final_measures
Processed: measures_pngs/Whisper-Not/final_measures
Processed: measures_pngs/All-Of-You/final_measures
Processed: measures_pngs/Moonglow/final_measures
Processed: measures_pngs/Satin-Doll/final_measures
Processed: measures_pngs/Meditation/final_measures
Pro